# 自作メタデータの検証

### 0. モデルをダウンロード

In [1]:
from huggingface_hub import snapshot_download
drive_path = "/home/paper_translator/data/models/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k"

In [2]:
download_path = snapshot_download(repo_id="mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k", local_dir=drive_path)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

### 1. セットアップ

In [3]:
import sys
from pprint import pprint
sys.path.append("/home/paper_translator/")
pprint(sys.path)

['/home/paper_translator/lib',
 '/home/paper_translator',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/home/paper_translator/.venv/lib/python3.11/site-packages',
 '/home/paper_translator/']


In [4]:
import logging
import sys

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [5]:
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
llmama_debug_handler = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llmama_debug_handler])

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


### 2. ドキュメントの準備

In [6]:
from llama_index import SimpleDirectoryReader
from src.XMLUtils import DocumentReader

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/paper_translator/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_verify_locations cafile='/home/paper_translator/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:slack_bolt.App:Sending a request - url: https://www.slack.com/api/auth.test, query_params: {}, body_params: {}, files: {}, json_body: None, headers: {'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': '(redacted)', 'User-Agent': 'Bolt/1.18.0 Python/3.11.5 slackclient/3.23.0 Linux/5.15.123.1-microsoft-standard-WSL2'}
DEBUG:slack_bolt.App:Received the following response - status: 200, headers: {'date': 'Mon, 09 Oct 2023 03:55:59 GMT', 'server': 'Apache', 'vary': 'Accept-Encoding', 'x-slack-req-id': 'f29ff8797f41377afce59ac2a4000dcd', 'x-content-type-options': 'nosniff', 'x-xss-protection': '0', 'pragma': 'no-cache', 'cache-control': 'private, no-cache

In [7]:
base_path = "/home/paper_translator/data"
document_name = (
    "Learning_Transferable_Visual_Models_From_Natural_Language_Supervision"
)
document_path = f"{base_path}/documents/{document_name}"
xml_path = f"{document_path}/{document_name}.tei.xml"

In [8]:
# Llama Index で提供されている SimpleDirectoryReader を使用して、
# ディレクトリ内の PDF ファイルをDocumentオブジェクトとして読み込む
required_exts = [".pdf"]
docs_1 = SimpleDirectoryReader(
    input_dir=document_path, required_exts=required_exts, recursive=True
).load_data()
print(f"documents_1 metadata: \n{docs_1[0].metadata}")

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 1
documents_1 metadata: 
{'page_label': '1', 'file_name': 'Learning_Transferable_Visual_Models_From_Natural_Language_Supervision.pdf'}


In [9]:
# 自作の DirectoryReader を使用して、
# ディレクトリ内の xml ファイルをDocumentオブジェクトとして読み込む
# run_grobid(dir_path, pdf_name)
docs_2 = DocumentReader().load_data(xml_path)
print(f"documents_2 metadata: \n{docs_2[0].metadata}")

/home/paper_translator/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


documents_2 metadata: 
{'Section No.': '1.', 'Section Title': 'Introduction and Motivating Work', 'Title': 'Learning Transferable Visual Models From Natural Language Supervision', 'All_Document_Summary': '\nState-of-the-art computer vision systems are trained to predict a fixed set of predetermined object categories. This restricted form of supervision limits their generality and usability since additional labeled data is needed to specify any other visual concept. Learning directly from raw text about images is a promising alternative which leverages a much broader source of supervision. We demonstrate that the simple pre-training task of predicting which caption goes with which image is an efficient and scalable way to learn SOTA image representations from scratch on a dataset of 400 million (image, text) pairs collected from the internet. After pre-training, natural language is used to reference learned visual concepts (or describe new ones) enabling zero-shot transfer of the model 

### 3. Context の設定

In [10]:
from llama_index import ServiceContext
from langchain.embeddings import HuggingFaceEmbeddings
from src.translator.llama_cpp import create_llama_cpp_model

In [11]:
model_path = "/home/paper_translator/data/models/ELYZA-japanese-Llama-2-7b-fast-instruct-q4_K_M.gguf"
llm = create_llama_cpp_model(package_name="llama_index", model_path=model_path)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1660 Ti, compute capability 7.5
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /home/paper_translator/data/models/ELYZA-japanese-Llama-2-7b-fast-instruct-q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 45043,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.f

In [42]:
from llama_index.schema import MetadataMode
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor, 
    SummaryExtractor, 
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    EntityExtractor
)

from llama_index.text_splitter import TokenTextSplitter

In [43]:
chunk_size = 3072
text_splitter = TokenTextSplitter(separator=' ', chunk_size=chunk_size, chunk_overlap=128)

In [44]:
metadata_extractor = MetadataExtractor(
    extractors=[
        #TitleExtractor(llm=llm),
        QuestionsAnsweredExtractor(questions=3, llm=llm),
        #SummaryExtractor(llm=llm, summaries=["prev", "self"]),
        KeywordExtractor(llm=llm),
        EntityExtractor(prediction_threshold=0.5)
    ],
    in_place=False,
)
node_parser = SimpleNodeParser(text_splitter=text_splitter, metadata_extractor=metadata_extractor)

DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /tomaarsen/span-marker-mbert-base-multinerd/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140716592797008 on /root/.cache/huggingface/hub/models--tomaarsen--span-marker-mbert-base-multinerd/blobs/c83d88707f975691482aeb63020ef15f18f4f366.lock
DEBUG:filelock:Lock 140716592797008 acquired on /root/.cache/huggingface/hub/models--tomaarsen--span-marker-mbert-base-multinerd/blobs/c83d88707f975691482aeb63020ef15f18f4f366.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /tomaarsen/span-marker-mbert-base-multinerd/resolve/main/config.json HTTP/1.1" 200 5083


DEBUG:filelock:Attempting to release lock 140716592797008 on /root/.cache/huggingface/hub/models--tomaarsen--span-marker-mbert-base-multinerd/blobs/c83d88707f975691482aeb63020ef15f18f4f366.lock
DEBUG:filelock:Lock 140716592797008 released on /root/.cache/huggingface/hub/models--tomaarsen--span-marker-mbert-base-multinerd/blobs/c83d88707f975691482aeb63020ef15f18f4f366.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /tomaarsen/span-marker-mbert-base-multinerd/resolve/main/adapter_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /tomaarsen/span-marker-mbert-base-multinerd/resolve/main/model.safetensors HTTP/1.1" 302 0
DEBUG:filelock:Attempting to acquire lock 140706966861008 on /root/.cache/huggingface/hub/models--tomaarsen--span-marker-mbert-base-multinerd/blobs/982693e472885f3bc1908b5a056c128fe69f1090deb8395df94167ba3eba1f04.lock
DEBUG:filelock:Lock 140706966861008 acquired on /root/.cache/huggingface/hub/models--tomaarsen--spa

DEBUG:filelock:Attempting to release lock 140706966861008 on /root/.cache/huggingface/hub/models--tomaarsen--span-marker-mbert-base-multinerd/blobs/982693e472885f3bc1908b5a056c128fe69f1090deb8395df94167ba3eba1f04.lock
DEBUG:filelock:Lock 140706966861008 released on /root/.cache/huggingface/hub/models--tomaarsen--span-marker-mbert-base-multinerd/blobs/982693e472885f3bc1908b5a056c128fe69f1090deb8395df94167ba3eba1f04.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140706968138064 on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/b122e74db13b415ea824c074da33c1c44f0d13a3.lock
DEBUG:filelock:Lock 140706968138064 acquired on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/b122e74db13b415ea824c074da33c1c44f0d13a3.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /bert-base-multilingual-cased/resolve/

DEBUG:filelock:Attempting to release lock 140706968138064 on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/b122e74db13b415ea824c074da33c1c44f0d13a3.lock
DEBUG:filelock:Lock 140706968138064 released on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/b122e74db13b415ea824c074da33c1c44f0d13a3.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140706968527312 on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/e3c6d456fb2616f01a9a6cd01a1be1a36353ed22.lock
DEBUG:filelock:Lock 140706968527312 acquired on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/e3c6d456fb2616f01a9a6cd01a1be1a36353ed22.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /bert-base-multilingual-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 29


DEBUG:filelock:Attempting to release lock 140706968527312 on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/e3c6d456fb2616f01a9a6cd01a1be1a36353ed22.lock
DEBUG:filelock:Lock 140706968527312 released on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/e3c6d456fb2616f01a9a6cd01a1be1a36353ed22.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/vocab.txt HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140706306012624 on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/e837bab60a5d204e29622d127c2dafe508aa0731.lock
DEBUG:filelock:Lock 140706306012624 acquired on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/e837bab60a5d204e29622d127c2dafe508aa0731.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /bert-base-multilingual-cased/resolve/main/vocab.txt HTTP/1.1" 200 995526


DEBUG:filelock:Attempting to release lock 140706306012624 on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/e837bab60a5d204e29622d127c2dafe508aa0731.lock
DEBUG:filelock:Lock 140706306012624 released on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/e837bab60a5d204e29622d127c2dafe508aa0731.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/tokenizer.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140706306012624 on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/420b0fc31334c64ddf53cc3e9222a6d4c59d0cae.lock
DEBUG:filelock:Lock 140706306012624 acquired on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/420b0fc31334c64ddf53cc3e9222a6d4c59d0cae.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /bert-base-multilingual-cased/resolve/main/tokenizer.json HTTP/1.1" 200 1961828


DEBUG:filelock:Attempting to release lock 140706306012624 on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/420b0fc31334c64ddf53cc3e9222a6d4c59d0cae.lock
DEBUG:filelock:Lock 140706306012624 released on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/420b0fc31334c64ddf53cc3e9222a6d4c59d0cae.lock
DEBUG:filelock:Lock 140706306012624 released on /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/blobs/420b0fc31334c64ddf53cc3e9222a6d4c59d0cae.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/added_tokens.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-multilingual-cased/resolve/main/special_tokens_map.json HTTP/1.1" 404 0


In [45]:
model_name = "sentence-transformers/all-MiniLM-l6-v2"
embed_model = HuggingFaceEmbeddings(model_name=model_name)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
    callback_manager=callback_manager,
    chunk_size=chunk_size,
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-l6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


### 4. Index の作成

In [13]:
# Summaryクエリ
SUMMARY_QUERY = "提供されたテキストの内容を要約してください。"

In [14]:
from llama_index import get_response_synthesizer
# from llama_index import VectorStoreIndex
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index.llms.base import ChatMessage, MessageRole
from llama_index.prompts import ChatPromptTemplate

# レスポンスシンセサイザーの準備
#response_synthesizer = get_response_synthesizer(
#    response_mode="tree_summarize",
#    use_async=True,
#)

In [ ]:
# DocumentSummaryIndexの準備
vector_store_index_1 = VectorStoreIndex.from_documents(
    docs_1,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
)

In [15]:
# (1) QAプロンプトの定義。
# QAシステムプロンプト
TEXT_QA_SYSTEM_PROMPT = ChatMessage(
    content=(
        "あなたは世界中で信頼されているQAシステムです。\n"
        "事前知識ではなく、常に提供されたコンテキスト情報を使用してクエリに回答してください。\n"
        "従うべきいくつかのルール:\n"
        "1. 回答内で指定されたコンテキストを直接参照しないでください。\n"
        "2. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。"
    ),
    role=MessageRole.SYSTEM,
)

# QAプロンプトテンプレートメッセージ
TEXT_QA_PROMPT_TMPL_MSGS = [
    TEXT_QA_SYSTEM_PROMPT,
    ChatMessage(
        content=(
            "コンテキスト情報は以下のとおりです。\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "事前知識ではなくコンテキスト情報を考慮して、クエリに答えます。\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
        role=MessageRole.USER,
    ),
]

# チャットQAプロンプト
CHAT_TEXT_QA_PROMPT = ChatPromptTemplate(message_templates=TEXT_QA_PROMPT_TMPL_MSGS)

In [16]:
# (2) TreeSummarizeプロンプトの定義。
# QAシステムプロンプト
TEXT_QA_SYSTEM_PROMPT = ChatMessage(
    content=(
        "あなたは世界中で信頼されているQAシステムです。\n"
        "事前知識ではなく、常に提供されたコンテキスト情報を使用してクエリに回答してください。\n"
        "従うべきいくつかのルール:\n"
        "1. 回答内で指定されたコンテキストを直接参照しないでください。\n"
        "2. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。"
    ),
    role=MessageRole.SYSTEM,
)

# ツリー要約プロンプトメッセージ
TREE_SUMMARIZE_PROMPT_TMPL_MSGS = [
    TEXT_QA_SYSTEM_PROMPT,
    ChatMessage(
        content=(
            "複数のソースからのコンテキスト情報を以下に示します。\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "予備知識ではなく、複数のソースからの情報を考慮して、質問に答えます。\n"
            "疑問がある場合は、「情報無し」と答えてください。\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
        role=MessageRole.USER,
    ),
]

# ツリー要約プロンプト
CHAT_TREE_SUMMARIZE_PROMPT = ChatPromptTemplate(
    message_templates=TREE_SUMMARIZE_PROMPT_TMPL_MSGS
)

In [17]:
from llama_index import StorageContext
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index.vector_stores import SimpleVectorStore

In [18]:
# Storage Context の作成
storage_context = StorageContext.from_defaults(
    docstore = SimpleDocumentStore(),
    vector_store = SimpleVectorStore(),
    index_store = SimpleIndexStore()
)

In [19]:
import nest_asyncio

# 非同期処理の有効化
nest_asyncio.apply()

# レスポンスシンセサイザーの準備
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    use_async=True,
    text_qa_template=CHAT_TEXT_QA_PROMPT,  # QAプロンプト
    summary_template=CHAT_TREE_SUMMARIZE_PROMPT,  # TreeSummarizeプロンプト
)

In [46]:
# DocumentSummaryIndexの準備
doc_summary_index = DocumentSummaryIndex.from_documents(
    docs_2,
    storage_context=storage_context,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
    summary_query=SUMMARY_QUERY,  # 要約クエリ
)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Pre-training methods which learn directly from ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: At the core of our approach is the idea of lear...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Existing work has mainly used three datasets, M...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: State-of-the-art computer vision systems use ve...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: We consider two different architectures for the...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: We train a series of 5 ResNets and 3 Vision Tra...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 3.1. Zero-Shot Transfer
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: In computer vision, zero-shot learning usually ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: CLIP is pre-trained to predict if an image and ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk

Extracting questions:   0%|          | 0/75 [00:00<?, ?it/s]

**********
Trace: index_construction
**********


KeyboardInterrupt: 

: 

In [13]:
query_engine_1 = vector_store_index_1.as_query_engine(service_context=service_context)
response_1 = query_engine_1.query("各セクションを翻訳してください。")
print(response_1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:llama_index.indices.utils:> Top 2 nodes:
> [Node 841fa9fa-4846-4e44-b537-c7f53e1ac44b] [Similarity score:             0.164076] 5 95.7 81.1 67.3 72.9 80.7 57.5 88.0 77.9 95.0 96.0 94.5 98.0 62.1 99.4 96.5 86.6 76.3 64.8 19.5 ...
> [Node 5017b3c1-9fac-4073-b19e-41e2569a855b] [Similarity score:             0.151905] Learning Transferable Visual Models From Natural Language Supervision 46
Text Retrieval Image Ret...


In [39]:
print(doc_summary_index.get_document_summary("3"))

提供されたテキストには、画像とテキストのペアからなるデータセットを使用して、自然言語による教師付き学習を行う方法についての説明が含まれています。この手法では、画像とキャプションの関連性を予測する事前学習タスクを使用して、画像表現を学習します。その後、学習されたビジュアルコンセプトを参照したり、新しいコンセプトを記述したりするために自然言語が使用されます。この手法は、OCR、動画のアクション認識、地理位置特定、細かいオブジェクト分類など、さまざまなコンピュータビジョンのタスクにおいて、既存のデータセットに特化したトレーニングなしでモデルを転移学習させることができます。また、この手法は、ImageNetなどのトレーニングデータを使用せずに、元のResNet-50と同等の精度を達成することができます。


In [23]:
from tqdm import tqdm

In [28]:
print(len(doc_summary_index.index_id))

36


In [40]:
for i in range(len(doc_summary_index.index_id)):
    print(f"{i}")
    try:
        print(doc_summary_index.get_document_summary(f"{i}"))
    except ValueError:
        print(f"doc_id: {i} is no text.")
        continue

0
提供されたテキストは、画像とテキストのペアからなるデータセットを使用して、自然言語を介して学習することで、画像表現を学習する方法について説明しています。この方法では、予め定義されたオブジェクトカテゴリの予測に限定されず、より広範な監督情報を利用することができます。具体的には、画像とキャプションの対応関係を予測する事前学習タスクを行い、その後、自然言語を使用して学習した視覚的な概念を参照したり、新しい概念を記述したりすることができます。このアプローチは、OCR、動画のアクション認識、地理位置情報、細かいオブジェクト分類など、さまざまなコンピュータビジョンのタスクにおいて、多くのタスクにおいて非自明な転移性を持ち、完全に監督されたベースラインと競合することができます。
1
doc_id: {i} is no text.
2
提供されたテキストによれば、最新のコンピュータビジョンシステムは、事前に定義された一連のオブジェクトカテゴリを予測するように訓練されています。しかし、この制限された形式の教示は、追加のラベル付きデータが必要であるため、一般性と使いやすさが制限されます。代わりに、画像に関する生のテキストから直接学習することは、より広範な教示源を活用する有望な方法です。このアプローチでは、どのキャプションがどの画像に対応するかを予測するという単純な事前トレーニングタスクを使用して、インターネットから収集された4億組の（画像、テキスト）ペアのデータセットを用いて、ゼロからSOTA（State-of-the-art）の画像表現を効率的かつスケーラブルに学習することを示しています。事前トレーニング後、自然言語を使用して学習された視覚的な概念を参照したり（または新しい概念を説明したり）することで、モデルをダウンストリームタスクにゼロショットで転送することができます。このアプローチの性能を評価するために、OCR、ビデオ内のアクション認識、地理位置特定、および多くのタイプの細かいオブジェクト分類などのタスクを含む30以上の既存のコンピュータビジョンデータセットでベンチマークを行っています。このモデルは、ほとんどのタスクにおいて非自明に転送され、データセット固有のトレーニングを必要とせずに完全に監視されたベースラインと競合することが多いです。たとえば、ImageNetのゼロショッ

In [5]:
from typing import List

import xml.etree.ElementTree as ET
from auto_gptq import AutoGPTQForCausalLM
from tqdm.auto import tqdm
from transformers import AutoTokenizer, pipeline
from xml.etree.ElementTree import Element

from src.OpenAIUtils import get_message

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/paper_translator/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:slack_bolt.App:Sending a request - url: https://www.slack.com/api/auth.test, query_params: {}, body_params: {}, files: {}, json_body: None, headers: {'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': '(redacted)', 'User-Agent': 'Bolt/1.18.0 Python/3.11.5 slackclient/3.23.0 Linux/5.15.123.1-microsoft-standard-WSL2'}
DEBUG:slack_bolt.App:Received the following response - status: 200, headers: {'date': 'Sun, 08 Oct 2023 08:38:23 GMT', 'server': 'Apache', 'vary': 'Accept-Encoding', 'x-slack-req-id': 'ee8f1ab48d5900c39ba3a01604cda71e', 'x-content-type-opt

In [6]:
class Section:
    def __init__(self, title: str = "", body: str = "") -> None:
        """セクションのタイトルと本文を保持するクラス
        Args:
            title (str, optional): セクションのタイトル. Defaults to "".
            body (str, optional): セクションの本文. Defaults to "".
        """
        self.title = title
        self.body = body
        
def get_text(element: Element):
    """XML要素からテキストを取得
    Args:
        element (Element): XMLの要素
    Returns:
        text: テキスト
    """
    text = ""
    for elem in element.iter():
        if elem.text:
            text += elem.text
        if elem.tail:
            text += elem.tail
    return text

def get_sections(root: Element) -> List[str]:
    """XMLからセクションを取得する関数
    Args:
        root (Element): XMLのルート要素
    Returns:
        sections (List[Section]): セクションのリスト
    """
    sections = []
    for div in root[1][0]:
        section = Section("", "")
        for element in div:
            if element.tag == "{http://www.tei-c.org/ns/1.0}head":
                section.title = element.text
            if element.tag == "{http://www.tei-c.org/ns/1.0}p":
                section.body += get_text(element=element)

        if section.body != "":
            sections.append(section)

    return sections

In [10]:
SYSTEM = """
### 指示 ###
論文の内容を理解した上で，重要なポイントを箇条書きで3点書いてください。
### 箇条書きの制約 ###
- 最大3個
- 日本語
- 箇条書き1個を100文字以内
### 対象とする論文の内容 ###
{text}
### 出力形式 ###
- 箇条書き1
- 箇条書き2
- 箇条書き3
"""

PROMPT_TEMPLATE = """
以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。
### 指示 ###
次の文章を翻訳してください。
{prompt_text}
### 応答: """


def write_markdown(
    sections: List[str],
    model,
    tokenizer,
    summarizer,
    ) -> str:
    """Markdownファイルを作成する関数
    Args:
        sections (List[str]): セクションのリスト
        model: GPTQモデル
        tokenizer: トークナイザー
        summarizer: サマライザー
    Returns:
        markdown_text (str): Markdownのテキスト
    """

    markdown_text = ""
    try:
        for section in tqdm(sections):
            # 144文字以下の場合は、全文を翻訳する
            if (len(section.body.split(" "))) < 144:
                # translated_text = translator(section.body)[0]["translation_text"]
                prompt_text = section.body
            # 144文字以上500文字以下の場合は、要約して翻訳する
            elif len(section.body.split(" ")) < 500:
                summary = summarizer(section.body)[0]["summary_text"]
                # translated_text = translator(summary)[0]["translation_text"]
                prompt_text = summary
            else:
                response = get_message(text=section.body, system=SYSTEM)
                # translated_text = translator(response)[0]["translation_text"]
                prompt_text = response

            tokens = (
                tokenizer(PROMPT_TEMPLATE.format(prompt_text=prompt_text), return_tensors="pt")
                .to("cuda:0")
                .input_ids
            )

            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token

            output = model.generate(
                input_ids=tokens,
                pad_token_id=tokenizer.pad_token_id,
                max_new_tokens=500,
                do_sample=True,
                temperature=0.1,
            )
            translated_text = tokenizer.decode(output[0])
            markdown_text += "\n" + translated_text

            if "conclusion" in section.title.lower():
                break
    except Exception as e:
        print(e)
        return markdown_text
    else:
        return markdown_text

In [8]:
model_dir_name="mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k"
# トークナイザーとモデルの準備
quantized_model_dir = model_dir_name
model_basename = "gptq_model-4bit-128g"

summarizer = pipeline("summarization", model="kworts/BARTxiv")
tokenizer = AutoTokenizer.from_pretrained(
    quantized_model_dir, use_fast=True
)

model = AutoGPTQForCausalLM.from_quantized(
    quantized_model_dir,
    model_basename=model_basename,
    use_safetensors=True,
    device="cuda:0",
    use_auth_token=False,
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /kworts/BARTxiv/resolve/main/adapter_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /kworts/BARTxiv/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /kworts/BARTxiv/resolve/main/generation_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k/resolve/main/config.json HTTP/1.1" 200 0


/home/paper_translator/.venv/lib/python3.11/site-packages/transformers/models/auto/configuration_auto.py:1020: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k/resolve/main/config.json HTTP/1.1" 200 0


/home/paper_translator/.venv/lib/python3.11/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k/resolve/main/quantize_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k/resolve/main/gptq_model-4bit-128g.safetensors HTTP/1.1" 302 0
INFO:auto_gptq.modeling._base:lm_head not been quantized, will be ignored when make_quant.


In [11]:
tree = ET.parse(xml_path)
root = tree.getroot()
sections = get_sections(root=root)
markdown_text = write_markdown(sections=sections, model=model, tokenizer=tokenizer, summarizer=summarizer)
# デバッグ用にテキストを保存する
with open(f"{document_path}/tmp_markdown.md", mode="w", encoding="utf-8") as f:
    f.write(markdown_text)

  0%|          | 0/44 [00:00<?, ?it/s]

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo-0301", "messages": [{"role": "system", "content": "\\n### \\u6307\\u793a ###\\n\\u8ad6\\u6587\\u306e\\u5185\\u5bb9\\u3092\\u7406\\u89e3\\u3057\\u305f\\u4e0a\\u3067\\uff0c\\u91cd\\u8981\\u306a\\u30dd\\u30a4\\u30f3\\u30c8\\u3092\\u7b87\\u6761\\u66f8\\u304d\\u30673\\u70b9\\u66f8\\u3044\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\n### \\u7b87\\u6761\\u66f8\\u304d\\u306e\\u5236\\u7d04 ###\\n- \\u6700\\u59273\\u500b\\n- \\u65e5\\u672c\\u8a9e\\n- \\u7b87\\u6761\\u66f8\\u304d1\\u500b\\u3092100\\u6587\\u5b57\\u4ee5\\u5185\\n### \\u5bfe\\u8c61\\u3068\\u3059\\u308b\\u8ad6\\u6587\\u306e\\u5185\\u5bb9 ###\\n{text}\\n### \\u51fa\\u529b\\u5f62\\u5f0f ###\\n- \\u7b87\\u6761\\u66f8\\u304d1\\n- \\u7b87\\u6761\\u66f8\\u304d2\\n- \\u7b87\\u6761\\u66f8\\u304d3\\n"}, {"role": "user", "content": "Pre-training methods which learn directly from raw

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=4737 request_id=920b8c24044546c3f39c9a5cccdd2b60 response_code=200
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=4737 request_id=920b8c24044546c3f39c9a5cccdd2b60 response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo-0301", "messages": [{"role": "system", "content": "\\n### \\u6307\\u793a ###\\n\\u8ad6\\u6587\\u306e\\u5185\\u5bb9\\u3092\\u7406\\u89e3\\u3057\\u305f\\u4e0a\\u3067\\uff0c\\u91cd\\u8981\\u306a\\u30dd\\u30a4\\u30f3\\u30c8\\u3092\\u7b87\\u6761\\u66f8\\u304d\\u30673\\u70b9\\u66f8\\u3044\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\n### \\u7b87\\u6761\\u66f8\\u304d\\u306e\\u5236\\u7d04 ###\\n